<a href="https://colab.research.google.com/github/felixsimard/comp551-p3/blob/main/Hassan_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pickle
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset
from google.colab import drive
from sklearn import preprocessing
from PIL import Image
from typing import List
from datetime import datetime

In [34]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Utility Functions

* Displays an image given a bit-based input

In [35]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(img)
    # plt.imshow(np.transpose(npimg, (1, 2, 0))) 
    plt.show()

* Class to create datasets with correct dimensionality properties, retreived from StackOverflow

In [36]:
# Reference: https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

* Export a CSV file in the parent directory given a pandas dataframe

In [42]:
def makeMyCSV(dataf: pd.DataFrame)->None:
  filename = 'kaggle_g19_{}.csv'.format(datetime.now())
  dataf.to_csv(filename, sep=',', float_format='{:36}', index=False)

In [39]:
# Felix's load data fn
# Function to return pickle loaded file in an ndarray
def load_data(filename, data_path='/content/drive/MyDrive/data/'):
    drive.mount("/content/drive")
    loaded_pkl = None
    try:
        pkl_buffered = open(data_path+''+filename,'rb')
        loaded_pkl = pickle.load(pkl_buffered)
    except Exception as e:
        print("Error loading data: {}".format(e))
    return loaded_pkl

In [40]:
def get_label_value(labels):
  """
  This function will return a string representing the label of a picture given
  the array label as input:
  Ex ouput: '1a', '4z' ...
  """
  label_temp = labels.tolist()
  label_temp = [int(x) for x in label_temp]
  number = label_temp[:10].index(1)
  letter = alpha_dict[label_temp[10:].index(1)]

  return str(number) + str(letter)

In [41]:
def transform_output(scores):
    """
    Input a Tensor and output will be another Tensor with same dimension but with all elements 0 except two.
    Those 2 elements will have value of 1 and will correspond to the models prediction about which letter and number
    is in the image.
    :param scores:
    :return:
    """
    return_array = []
    score_list = scores.tolist()

    for score in score_list:
        numbers = score[:10]
        letters = score[10:]
        test = lambda x, max_value : 1 if x >= max_value else 0

        new_numbers = [test(x, max(numbers)) for x in numbers]
        new_letters = [test(x, max(letters)) for x in letters]

        return_array.append(new_numbers + new_letters)

    return return_array

## Pickle Data to Numpy NDArray

In [44]:
# loading all data
train_features = load_data("images_l.pkl")[:, None]
train_labels = load_data("labels_l.pkl")
test = load_data("images_test.pkl")[:, None]
train_unlabelled = load_data("images_ul.pkl")[:, None]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
print(train_features.shape, train_features[:1])
print(train_labels.shape, train_labels[:1])

(30000, 56, 56) [[[  0.   0.   0. ... 175.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0. 175.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]]
(30000, 36) [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


- `train_features` has 30,000 samples of 56x56 images
- `train_labels` labels of the 56x56 images, a 36-bit binary vector
- The code block below verifies the image data are all in numpy n-dimensional arrays, `np.ndarray`

In [7]:
for data in [train_features, train_labels, train_unlabelled, test]:  
  print(type(data) is np.ndarray)

True
True
True
True


## Hyperparameters

In [45]:
epochs = 50
batch = 16
lr = 0.002             # learning rate
channels = 1           # Image input channel
train_test_split = 0.3 # 70% training data, 30% validation data
flatten_dim = 56 * 56  # image dim = 56 x 56 px

## Training & Validation Split

In [49]:
split_index = math.floor(len(train_labels)*train_test_split)

full_train_l = train_features
val_l = train_features[:split_index]
train_features = train_features[split_index:]

full_train_labels_l = train_labels_l
val_labels_l = train_labels_l[:split_index]
train_labels_l = train_labels_l[split_index:]

# DataLoaders
full_train_l_dataloader = DataLoader(MyDataset(,, transform=Transform), shuffle=True, batch_size=batch)
train_l_dataloader = DataLoader(MyDataset(train_l, train_labels_l, transform=transform), shuffle=True, batch_size=BATCH_SIZE)
val_l_dataloader = DataLoader(MyDataset(val_l, val_labels_l, transform=transform), shuffle=True)

# Test set for Kaggle
test_labels_ul = np.zeros(len(test_ul))
test_ul_dataloader = DataLoader(MyDataset(test_ul, test_labels_ul, transform=transform), batch_size=BATCH_SIZE, shuffle=False)

SyntaxError: ignored

## Tensor DataLoader & Feature Labels

In [48]:
# Data transformation parameters
mean = (0.5,)
std = (0.5,)
Transform = transforms.Compose([transforms.Normalize(mean=mean, std=std)])

In [16]:
train_labels = train_labels.tolist()
# Transforming the numpy arrays into tensors with the labels
# Concatenating datasets to have Tensor([[image_features], label])
training = DataLoader(TensorDataset(torch.Tensor(train_features).unsqueeze(1),
                                    torch.Tensor(train_labels).unsqueeze(1)), batch_size=batch, shuffle=False)

test_labels = torch.Tensor(np.zeros(len(test)))
testing = DataLoader(TensorDataset(torch.Tensor(test), test_labels))

print(len(training)*batch,len(testing))

30000 15000


- The classification task calls for classifying an image that contains:
1. Characters `A-Z` OR `a-z`
2. Numbers `0-9`
- Each image will include any combination of 1 lower OR uppercase character and one number
- Therefore, the labels will have to include every combination of these characters and numbers:
1. 260 different classes: `0-9` AND `A-Z`
2. 260 different classes: `0-9` AND `a-z`
- A total of 520 `labels`

## Conv. NN Class (Implementation of VGG11 Deep CNN)

In [24]:
class CNN(nn.Module):

  # Constructor
  def __init__(self, in_channels=1, num_classes=36):
    super(CNN, self).__init__()         # Access methods in parent class
    self.in_channels = in_channels
    self.num_classes = num_classes
    # convolutional layers 
    self.conv_layers = nn.Sequential(
      nn.Conv2d(self.in_channels, 64, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(128, 256, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(256, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
      )
        # fully connected linear layers
    self.linear_layers = nn.Sequential(
      nn.Linear(in_features=512, out_features=4096),
      nn.ReLU(),
      nn.Dropout2d(0.5),
      nn.Linear(in_features=4096, out_features=4096),
      nn.ReLU(),
      nn.Dropout2d(0.5),
      nn.Linear(in_features=4096, out_features=self.num_classes)
      )
    
  def forward(self, x):
      x = self.conv_layers(x)
      # flatten to prepare for the fully connected layers
      x = x.view(x.size(0),-1)
      x = self.linear_layers(x)
      return x
  

# Model Loss, Optimization, & Run with CUDA



In [26]:
model = CNN().to(device)
criterion = nn.MultiMarginLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
steps = len(training)

cpu


In [27]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(training, 0):
        inputs, labels = data[0].to(device), data[1].squeeze_().long().to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i+1) % 100 == 0: # print every 1000 mini-batches
            print(f'Epoch [{epoch+1}/ {epochs}], Step [{i+1}/{len(training)}], {running_loss/2000}')
            running_loss=0
        
torch.save(model.state_dict(), './cnn.pth')

RuntimeError: ignored

In [ ]:
df = pd.DataFrame(columns=['# Id', 'Category'])
#device = torch.device('cpu')
with torch.no_grad():
  i=0
  for data in testing:
    images,labels = data
    images = data[0].to(device)[None, :]
    images = images.permute(1,0,2, 3)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    label_predicted = labels_encoder.inverse_transform(predicted.cpu().numpy())
    prediction = str(label_predicted[0])
    df.loc[i] = [i, prediction]
    i += 1

df = df.iloc[:15001]

df

,# Id,Category
0,0,000001000000010000000000000000000000
1,1,000000000100000000000010000000000000
2,2,001000000000000000000100000000000000
3,3,000100000000000000000000000000000010
4,4,001000000000000000000100000000000000
...,...,...
14995,14995,000000000110000000000000000000000000
14996,14996,000100000000000000000000000000000001
14997,14997,000000001000000000000001000000000000
14998,14998,000100000000000000000000000000000001
